# Classement des valeurs de la BRVM par capitalisation boursière

Ce notebook permet de classer les valeurs cotées à la BRVM par capitalisation boursière, et d'enrichir ce classement avec des informations financières clés :
- Cours actuel de l'action
- PER (Price-to-Earnings Ratio) de 2020 à 2024
- Dividende par action de 2020 à 2024
- Ratio dividende 2024 / cours actuel (rendement du dividende)

Les données sont extraites du site de Sika Finance, en particulier de la section "cotations de A à Z" et des pages spécifiques à chaque société.

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import HTML

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12
sns.set_palette("viridis")

# Désactiver les avertissements
import warnings
warnings.filterwarnings('ignore')

## 1. Scraping des données de Sika Finance

Nous allons d'abord extraire la liste des valeurs cotées à la BRVM depuis le site de Sika Finance, puis collecter pour chaque valeur les informations financières demandées.

In [ ]:
def get_session():
    """Initialise et renvoie une session HTTP avec les entêtes appropriés."""
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'
    })
    return session

def get_brvm_values(session):
    """Récupère la liste des valeurs cotées à la BRVM depuis Sika Finance."""
    url = "https://www.sikafinance.com/marches/cotations-brvm"
    
    try:
        response = session.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Chercher le tableau des cotations
        table = soup.find('table', class_='table-cotation')
        
        if not table:
            print("Tableau des cotations non trouvé.")
            return []
        
        values = []
        rows = table.find_all('tr')[1:]  # Ignorer l'en-tête
        
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 9:  # Vérifier qu'il y a assez de cellules
                # Extraire les informations de base
                try:
                    symbol = cells[0].text.strip()
                    name = cells[1].text.strip()
                    sector = cells[2].text.strip()
                    current_price = float(cells[3].text.strip().replace(' ', '').replace(',', '.') or 0)
                    change = cells[4].text.strip().replace('%', '').replace(',', '.').replace(' ', '')
                    change = float(change) if change else 0
                    volume = int(cells[5].text.strip().replace(' ', '') or 0)
                    previous_price = float(cells[6].text.strip().replace(' ', '').replace(',', '.') or 0)
                    year_high = float(cells[7].text.strip().replace(' ', '').replace(',', '.') or 0)
                    year_low = float(cells[8].text.strip().replace(' ', '').replace(',', '.') or 0)
                    
                    # Si possible, récupérer l'URL de la page de détail
                    detail_link = None
                    if cells[0].find('a'):
                        detail_link = cells[0].find('a').get('href')
                        if detail_link and not detail_link.startswith('http'):
                            detail_link = f"https://www.sikafinance.com{detail_link}"
                    
                    values.append({
                        'symbol': symbol,
                        'name': name,
                        'sector': sector,
                        'current_price': current_price,
                        'change': change,
                        'volume': volume,
                        'previous_price': previous_price,
                        'year_high': year_high,
                        'year_low': year_low,
                        'detail_link': detail_link
                    })
                except Exception as e:
                    print(f"Erreur lors de l'extraction des données pour une valeur: {str(e)}")
                    continue
        
        print(f"Récupéré {len(values)} valeurs cotées à la BRVM.")
        return values
    
    except Exception as e:
        print(f"Erreur lors de la récupération des valeurs: {str(e)}")
        return []

def get_market_cap(session, symbol, current_price):
    """Essaie de récupérer la capitalisation boursière pour une valeur donnée."""
    url = f"https://www.sikafinance.com/marches/cotation_seance/{symbol}"
    
    try:
        response = session.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Chercher les informations de capitalisation boursière
        market_cap = None
        
        # Méthode 1 : chercher directement dans la page
        cap_elements = soup.find_all(text=re.compile('Capitalisation', re.IGNORECASE))
        
        for element in cap_elements:
            parent = element.parent
            if parent:
                next_sibling = parent.find_next_sibling()
                if next_sibling:
                    cap_text = next_sibling.text.strip()
                    # Extraire les chiffres
                    cap_value = re.sub(r'[^\d,]', '', cap_text)
                    if cap_value:
                        try:
                            market_cap = float(cap_value.replace(',', '.'))
                            # Convertir en milliards si nécessaire
                            if 'milliard' in cap_text.lower():
                                market_cap *= 1e9
                            elif 'million' in cap_text.lower():
                                market_cap *= 1e6
                            break
                        except:
                            pass
        
        # Si pas trouvé, méthode 2 : chercher le nombre d'actions
        if not market_cap:
            shares_elements = soup.find_all(text=re.compile('Nombre d\'actions', re.IGNORECASE))
            
            for element in shares_elements:
                parent = element.parent
                if parent:
                    next_sibling = parent.find_next_sibling()
                    if next_sibling:
                        shares_text = next_sibling.text.strip()
                        # Extraire les chiffres
                        shares_value = re.sub(r'[^\d]', '', shares_text)
                        if shares_value:
                            try:
                                shares = int(shares_value)
                                market_cap = shares * current_price
                                break
                            except:
                                pass
        
        return market_cap
    
    except Exception as e:
        print(f"Erreur lors de la récupération de la capitalisation pour {symbol}: {str(e)}")
        return None

def get_financial_data(session, symbol):
    """Récupère les données financières (PER, dividendes) pour une valeur donnée."""
    url = f"https://www.sikafinance.com/bourse/societe/{symbol}"
    
    financial_data = {
        'per_2020': None,
        'per_2021': None,
        'per_2022': None,
        'per_2023': None,
        'per_2024': None,
        'div_2020': None,
        'div_2021': None,
        'div_2022': None,
        'div_2023': None,
        'div_2024': None
    }
    
    try:
        response = session.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Chercher les tableaux de données financières
        tables = soup.find_all('table')
        
        for table in tables:
            # Chercher les données de PER
            if 'PER' in table.text or 'P/E' in table.text or 'Price Earning Ratio' in table.text:
                rows = table.find_all('tr')
                for row in rows:
                    cells = row.find_all('td')
                    if len(cells) >= 2:
                        header = cells[0].text.strip()
                        if '2020' in header:
                            per_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['per_2020'] = float(per_text) if per_text and per_text != '-' else None
                            except:
                                pass
                        elif '2021' in header:
                            per_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['per_2021'] = float(per_text) if per_text and per_text != '-' else None
                            except:
                                pass
                        elif '2022' in header:
                            per_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['per_2022'] = float(per_text) if per_text and per_text != '-' else None
                            except:
                                pass
                        elif '2023' in header:
                            per_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['per_2023'] = float(per_text) if per_text and per_text != '-' else None
                            except:
                                pass
                        elif '2024' in header:
                            per_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['per_2024'] = float(per_text) if per_text and per_text != '-' else None
                            except:
                                pass
            
            # Chercher les données de dividende
            if 'Dividende' in table.text or 'DPA' in table.text or 'Div/Action' in table.text:
                rows = table.find_all('tr')
                for row in rows:
                    cells = row.find_all('td')
                    if len(cells) >= 2:
                        header = cells[0].text.strip()
                        if '2020' in header:
                            div_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['div_2020'] = float(div_text) if div_text and div_text != '-' else None
                            except:
                                pass
                        elif '2021' in header:
                            div_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['div_2021'] = float(div_text) if div_text and div_text != '-' else None
                            except:
                                pass
                        elif '2022' in header:
                            div_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['div_2022'] = float(div_text) if div_text and div_text != '-' else None
                            except:
                                pass
                        elif '2023' in header:
                            div_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['div_2023'] = float(div_text) if div_text and div_text != '-' else None
                            except:
                                pass
                        elif '2024' in header:
                            div_text = cells[1].text.strip().replace(',', '.').replace(' ', '')
                            try:
                                financial_data['div_2024'] = float(div_text) if div_text and div_text != '-' else None
                            except:
                                pass
        
        return financial_data
    
    except Exception as e:
        print(f"Erreur lors de la récupération des données financières pour {symbol}: {str(e)}")
        return financial_data

In [ ]:
# Initialiser la session HTTP
session = get_session()

# Récupérer la liste des valeurs cotées
values = get_brvm_values(session)

# Créer un DataFrame avec les valeurs de base
df_values = pd.DataFrame(values)

# Afficher les premières lignes pour vérifier
df_values.head()

## 2. Récupération des capitalisations boursières

In [ ]:
# Récupérer les capitalisations boursières
market_caps = []
for index, row in df_values.iterrows():
    symbol = row['symbol']
    current_price = row['current_price']
    
    print(f"Récupération de la capitalisation boursière pour {symbol}...")
    market_cap = get_market_cap(session, symbol, current_price)
    market_caps.append(market_cap)
    
    # Respecter une pause pour ne pas surcharger le serveur
    time.sleep(1)

# Ajouter les capitalisations au DataFrame
df_values['market_cap'] = market_caps

# Afficher les valeurs avec leur capitalisation
df_values[['symbol', 'name', 'current_price', 'market_cap']].head(10)

## 3. Récupération des données financières (PER et dividendes)

In [ ]:
# Récupérer les données financières pour chaque valeur
financial_data_list = []
for index, row in df_values.iterrows():
    symbol = row['symbol']
    
    print(f"Récupération des données financières pour {symbol}...")
    financial_data = get_financial_data(session, symbol)
    financial_data_list.append(financial_data)
    
    # Respecter une pause pour ne pas surcharger le serveur
    time.sleep(1)

# Convertir la liste en DataFrame
df_financial = pd.DataFrame(financial_data_list)

# Joindre les deux DataFrames
df_combined = pd.concat([df_values, df_financial], axis=1)

# Afficher les premières lignes pour vérifier
df_combined.head()

## 4. Calcul du rendement du dividende

In [ ]:
# Calculer le rendement du dividende (dividende 2024 / cours actuel)
df_combined['dividend_yield'] = df_combined.apply(
    lambda row: (row['div_2024'] / row['current_price'] * 100) if row['div_2024'] is not None and row['current_price'] > 0 else None,
    axis=1
)

# Afficher le tableau avec le rendement du dividende
columns_to_display = [
    'symbol', 'name', 'sector', 'current_price', 'market_cap', 
    'per_2020', 'per_2021', 'per_2022', 'per_2023', 'per_2024',
    'div_2020', 'div_2021', 'div_2022', 'div_2023', 'div_2024',
    'dividend_yield'
]

df_display = df_combined[columns_to_display].copy()

# Arrondir les valeurs numériques pour une meilleure lisibilité
for col in df_display.columns:
    if col not in ['symbol', 'name', 'sector']:
        df_display[col] = df_display[col].round(2)

# Afficher le tableau
df_display.head(10)

## 5. Classement par capitalisation boursière

In [ ]:
# Trier le DataFrame par capitalisation boursière décroissante
df_sorted = df_display.sort_values(by='market_cap', ascending=False).reset_index(drop=True)

# Afficher le top 20 des valeurs par capitalisation
df_sorted.head(20)

## 6. Visualisations

### 6.1 Top 10 des capitalisations boursières

In [ ]:
# Sélectionner le top 10 pour la visualisation
top_10 = df_sorted.head(10).copy()

# Créer un graphique à barres pour les capitalisations
plt.figure(figsize=(16, 8))
ax = sns.barplot(x='symbol', y='market_cap', data=top_10)
plt.title('Top 10 des valeurs de la BRVM par capitalisation boursière', fontsize=16)
plt.xlabel('Symbole')
plt.ylabel('Capitalisation boursière (FCFA)')
plt.xticks(rotation=45)

# Ajouter les valeurs sur les barres
for i, v in enumerate(top_10['market_cap']):
    if not pd.isna(v):
        if v >= 1e9:
            label = f"{v/1e9:.2f} Mds"
        elif v >= 1e6:
            label = f"{v/1e6:.2f} M"
        else:
            label = f"{v:.0f}"
        ax.text(i, v * 1.02, label, ha='center', fontsize=10, rotation=45)

plt.tight_layout()
plt.show()

### 6.2 Rendement du dividende par valeur

In [ ]:
# Sélectionner les 15 valeurs avec les meilleurs rendements de dividendes
top_div_yield = df_sorted[df_sorted['dividend_yield'].notna()].sort_values(by='dividend_yield', ascending=False).head(15)

# Créer un graphique à barres pour les rendements de dividendes
plt.figure(figsize=(16, 8))
ax = sns.barplot(x='symbol', y='dividend_yield', data=top_div_yield)
plt.title('Top 15 des valeurs de la BRVM par rendement du dividende', fontsize=16)
plt.xlabel('Symbole')
plt.ylabel('Rendement du dividende (%)')
plt.xticks(rotation=45)

# Ajouter les valeurs sur les barres
for i, v in enumerate(top_div_yield['dividend_yield']):
    if not pd.isna(v):
        ax.text(i, v * 1.02, f"{v:.2f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

### 6.3 Évolution du PER entre 2020 et 2024

In [ ]:
# Sélectionner les 10 premières valeurs par capitalisation avec des données de PER disponibles
top_per = df_sorted.head(10).copy()

# Préparer les données pour le graphique
per_data = []
for index, row in top_per.iterrows():
    for year in [2020, 2021, 2022, 2023, 2024]:
        per_value = row[f'per_{year}']
        if not pd.isna(per_value):
            per_data.append({
                'symbol': row['symbol'],
                'year': year,
                'per': per_value
            })

per_df = pd.DataFrame(per_data)

# Créer le graphique d'évolution du PER
if not per_df.empty:
    plt.figure(figsize=(16, 8))
    sns.lineplot(x='year', y='per', hue='symbol', data=per_df, marker='o')
    plt.title('Évolution du PER des principales valeurs (2020-2024)', fontsize=16)
    plt.xlabel('Année')
    plt.ylabel('PER')
    plt.xticks(rotation=0)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(title='Symbole', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("Pas assez de données de PER disponibles pour créer le graphique.")

### 6.4 Évolution des dividendes entre 2020 et 2024

In [ ]:
# Sélectionner les 10 premières valeurs par capitalisation avec des données de dividende disponibles
top_div = df_sorted.head(10).copy()

# Préparer les données pour le graphique
div_data = []
for index, row in top_div.iterrows():
    for year in [2020, 2021, 2022, 2023, 2024]:
        div_value = row[f'div_{year}']
        if not pd.isna(div_value):
            div_data.append({
                'symbol': row['symbol'],
                'year': year,
                'dividend': div_value
            })

div_df = pd.DataFrame(div_data)

# Créer le graphique d'évolution des dividendes
if not div_df.empty:
    plt.figure(figsize=(16, 8))
    sns.lineplot(x='year', y='dividend', hue='symbol', data=div_df, marker='o')
    plt.title('Évolution des dividendes des principales valeurs (2020-2024)', fontsize=16)
    plt.xlabel('Année')
    plt.ylabel('Dividende par action (FCFA)')
    plt.xticks(rotation=0)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(title='Symbole', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("Pas assez de données de dividendes disponibles pour créer le graphique.")

## 7. Création d'une visualisation HTML interactive

In [ ]:
# Créer un tableau de bord interactif avec Plotly
def create_interactive_dashboard(df):
    # Filtrer les valeurs avec capitalisation boursière disponible
    df_filtered = df[df['market_cap'].notna()].copy()
    
    # Top 15 des capitalisations
    top_market_cap = df_filtered.sort_values(by='market_cap', ascending=False).head(15)
    
    # Créer le graphique interactif des capitalisations
    fig1 = px.bar(
        top_market_cap, 
        x='symbol', 
        y='market_cap',
        title='Top 15 des valeurs par capitalisation boursière',
        hover_data=['name', 'sector', 'current_price'],
        color='market_cap',
        color_continuous_scale='Blues',
        labels={
            'symbol': 'Symbole',
            'market_cap': 'Capitalisation boursière (FCFA)',
            'name': 'Nom',
            'sector': 'Secteur',
            'current_price': 'Cours actuel'
        }
    )
    
    fig1.update_layout(
        xaxis_tickangle=-45,
        autosize=True,
        height=500,
        margin=dict(l=50, r=50, b=100, t=100, pad=4)
    )
    
    # Préparer les données pour le graphique PER
    per_data = []
    symbols = top_market_cap['symbol'].tolist()
    
    for symbol in symbols:
        row = df[df['symbol'] == symbol].iloc[0]
        for year in [2020, 2021, 2022, 2023, 2024]:
            per_value = row[f'per_{year}']
            if not pd.isna(per_value):
                per_data.append({
                    'symbol': symbol,
                    'name': row['name'],
                    'year': year,
                    'per': per_value
                })
    
    per_df = pd.DataFrame(per_data)
    
    # Graphique d'évolution du PER
    if not per_df.empty:
        fig2 = px.line(
            per_df, 
            x='year', 
            y='per', 
            color='symbol',
            title='Évolution du PER des principales valeurs (2020-2024)',
            markers=True,
            hover_data=['name'],
            labels={
                'year': 'Année',
                'per': 'PER',
                'symbol': 'Symbole',
                'name': 'Nom'
            }
        )
        
        fig2.update_layout(
            autosize=True,
            height=500,
            margin=dict(l=50, r=50, b=50, t=100, pad=4)
        )
    else:
        fig2 = go.Figure()
        fig2.update_layout(
            title="Pas assez de données de PER disponibles",
            annotations=[dict(
                text="Données insuffisantes",
                xref="paper",
                yref="paper",
                x=0.5,
                y=0.5,
                showarrow=False
            )]
        )
    
    # Top 15 des rendements de dividendes
    top_div_yield = df_filtered[df_filtered['dividend_yield'].notna()].sort_values(by='dividend_yield', ascending=False).head(15)
    
    # Graphique des rendements de dividendes
    fig3 = px.bar(
        top_div_yield, 
        x='symbol', 
        y='dividend_yield',
        title='Top 15 des valeurs par rendement du dividende',
        hover_data=['name', 'sector', 'current_price', 'div_2024'],
        color='dividend_yield',
        color_continuous_scale='Greens',
        labels={
            'symbol': 'Symbole',
            'dividend_yield': 'Rendement du dividende (%)',
            'name': 'Nom',
            'sector': 'Secteur',
            'current_price': 'Cours actuel',
            'div_2024': 'Dividende 2024'
        }
    )
    
    fig3.update_layout(
        xaxis_tickangle=-45,
        autosize=True,
        height=500,
        margin=dict(l=50, r=50, b=100, t=100, pad=4)
    )
    
    # Scatter plot PER vs Dividend Yield
    df_scatter = df_filtered[(df_filtered['per_2024'].notna()) & (df_filtered['dividend_yield'].notna())].copy()
    
    if not df_scatter.empty:
        fig4 = px.scatter(
            df_scatter, 
            x='per_2024', 
            y='dividend_yield',
            size='market_cap',
            color='sector',
            hover_name='symbol',
            hover_data=['name', 'current_price'],
            title='Relation entre PER 2024 et rendement du dividende',
            labels={
                'per_2024': 'PER 2024',
                'dividend_yield': 'Rendement du dividende (%)',
                'market_cap': 'Capitalisation boursière',
                'sector': 'Secteur',
                'name': 'Nom',
                'current_price': 'Cours actuel'
            }
        )
        
        fig4.update_layout(
            autosize=True,
            height=500,
            margin=dict(l=50, r=50, b=50, t=100, pad=4)
        )
    else:
        fig4 = go.Figure()
        fig4.update_layout(
            title="Données insuffisantes pour la relation PER-Dividende",
            annotations=[dict(
                text="Données insuffisantes",
                xref="paper",
                yref="paper",
                x=0.5,
                y=0.5,
                showarrow=False
            )]
        )
    
    # Combiner les graphiques en HTML
    html_content = f"""
    <!DOCTYPE html>
    <html lang="fr">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Classement des valeurs de la BRVM</title>
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
        <style>
            body {{ font-family: Arial, sans-serif; margin: 0; padding: 20px; background-color: #f8f9fa; }}
            .container {{ max-width: 1400px; margin: 0 auto; }}
            .header {{ text-align: center; margin-bottom: 30px; }}
            .card {{ margin-bottom: 20px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); }}
            .chart-container {{ background-color: white; padding: 15px; border-radius: 5px; }}
            h1, h2 {{ color: #0d6efd; }}
            .table-container {{ overflow-x: auto; }}
            table {{ width: 100%; border-collapse: collapse; margin-bottom: 20px; }}
            th, td {{ padding: 12px 15px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #f8f9fa; color: #495057; font-weight: bold; }}
            tr:hover {{ background-color: #f8f9fa; }}
            .positive {{ color: #198754; }}
            .negative {{ color: #dc3545; }}
            .footer {{ text-align: center; margin-top: 30px; padding: 10px; color: #6c757d; }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>Classement des valeurs de la BRVM</h1>
                <p class="text-muted">Données extraites le {datetime.now().strftime('%d/%m/%Y')}</p>
            </div>
            
            <div class="row mb-4">
                <div class="col-12">
                    <div class="card">
                        <div class="card-header">
                            <h2>Tableau complet des valeurs classées par capitalisation boursière</h2>
                        </div>
                        <div class="card-body table-container">
                            {df_sorted.to_html(classes='table table-striped table-hover', index=False, na_rep='-')}
                        </div>
                    </div>
                </div>
            </div>
            
            <div class="row mb-4">
                <div class="col-12">
                    <div class="card">
                        <div class="card-header">
                            <h2>Top 15 des valeurs par capitalisation boursière</h2>
                        </div>
                        <div class="card-body chart-container">
                            {fig1.to_html(full_html=False, include_plotlyjs='cdn')}
                        </div>
                    </div>
                </div>
            </div>
            
            <div class="row mb-4">
                <div class="col-12">
                    <div class="card">
                        <div class="card-header">
                            <h2>Top 15 des valeurs par rendement du dividende</h2>
                        </div>
                        <div class="card-body chart-container">
                            {fig3.to_html(full_html=False, include_plotlyjs='cdn')}
                        </div>
                    </div>
                </div>
            </div>
            
            <div class="row mb-4">
                <div class="col-md-6">
                    <div class="card h-100">
                        <div class="card-header">
                            <h2>Évolution du PER (2020-2024)</h2>
                        </div>
                        <div class="card-body chart-container">
                            {fig2.to_html(full_html=False, include_plotlyjs='cdn')}
                        </div>
                    </div>
                </div>
                <div class="col-md-6">
                    <div class="card h-100">
                        <div class="card-header">
                            <h2>Relation PER vs Rendement du dividende</h2>
                        </div>
                        <div class="card-body chart-container">
                            {fig4.to_html(full_html=False, include_plotlyjs='cdn')}
                        </div>
                    </div>
                </div>
            </div>
            
            <div class="footer">
                <p>© {datetime.now().year} - Analyse des valeurs de la BRVM</p>
            </div>
        </div>
        
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js"></script>
    </body>
    </html>
    """
    
    return html_content

# Créer le tableau de bord interactif
html_dashboard = create_interactive_dashboard(df_sorted)

# Sauvegarder le tableau de bord dans un fichier HTML
html_file = f"../dashboard/classement_brvm_{datetime.now().strftime('%Y%m%d')}.html"

# Créer le répertoire dashboard s'il n'existe pas
import os
if not os.path.exists('../dashboard'):
    os.makedirs('../dashboard')

with open(html_file, 'w', encoding='utf-8') as f:
    f.write(html_dashboard)

print(f"Tableau de bord sauvegardé dans : {html_file}")

# Afficher le tableau de bord dans le notebook
HTML(html_dashboard)

## 8. Conclusion

Dans ce notebook, nous avons réalisé un classement complet des valeurs cotées à la BRVM par capitalisation boursière. Nous avons collecté et analysé les données suivantes :

- Le cours actuel de chaque action
- La capitalisation boursière
- Le PER (Price-to-Earnings Ratio) de 2020 à 2024
- Le dividende par action de 2020 à 2024
- Le rendement du dividende (ratio dividende 2024 / cours actuel)

Les principales observations sont les suivantes :

1. Les valeurs à plus forte capitalisation sont principalement dans les secteurs [à compléter selon les résultats].
2. Les valeurs offrant les meilleurs rendements de dividendes sont [à compléter selon les résultats].
3. L'évolution du PER montre [à compléter selon les résultats].
4. La relation entre PER et rendement du dividende indique [à compléter selon les résultats].

Le tableau de bord HTML interactif permet une exploration plus approfondie de ces données et facilite la prise de décision pour les investisseurs intéressés par le marché de la BRVM.